In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 데이터를 저장할 빈 배열
data = []

# 크롤링할 페이지 수를 사용자에게 입력받기
page = int(input("몇 페이지까지 크롤링 할까요? "))

# 검색어를 사용자에게 입력받기
searchQuery = input("검색어를 입력하세요")


# 사용자가 입력한 페이지 수만큼 반복
for i in range(1, page+1):
    # 네이트 뉴스 검색 페이지에 접근
    response = requests.get(f"https://news.nate.com/search?q={searchQuery}&page={i}")
    # HTML 코드를 파싱
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    # 뉴스 기사 리스트를 선택
    posts = soup.select(".search-list > li")
    # 각 뉴스 기사에 대해 반복
    for post in posts:
        # 뉴스 기사 제목을 추출
        title = post.select_one(".thumb-wrap > .info > .info > .tit")
        # 에러 처리
        if title is None:
            continue
        # 뉴스 기사 링크를 추출
        link = post.select_one(".thumb-wrap")
        # 에러 처리
        if link is None:
            continue
        # 뉴스 기사 언론사를 추출
        source = post.select_one(".time")
        # 에러 처리
        if source is None:
            continue
        # 뉴스 기사 작성날짜를 추출
        date = source.text.strip().split('\n')[1].strip()
        # 추출한 데이터를 배열에 추가
        data.append([title.text.strip(), link.attrs['href'].strip().replace('//', 'https://'), source.text.strip().split('\n')[0].strip(), date])

    # 다음 버튼이 없는 경우 중지    
    if soup.find('a', {'title': '다음'}) == None:
        break

# pandas 프레임으로 저장
df = pd.DataFrame(data, columns=['제목', '링크', '언론사', '작성날짜'])
df.to_excel("nate_news.xlsx", index=False)